# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 2 [10 баллов] до 23:59 22.03.2018. Предсказание цены акции по экономическим новостям


В этом домашнем задании вы попытаетесь предсказать рост цены акции компании Газпром по новостям о компании. Домашнее задание состоит из трех частей:
1. Предварительная обработка текстов и эксплоративный анализ
2. Baseline алгоритм
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



Входные данные:
* Новости о компании "Газпром", начиная с 2010 года
* Стоимость акций компании "Газпром" на ММВБ, начиная с 2010 года
    * цена открытия (Open)
    * цена закрытия (ClosingPrice)
    * максимальная цена за день (DailyHigh)
    * минимальная цена за день (DailyLow) 
    * объем бумаг (VolumePcs)


In [292]:
import numpy as np
import pandas as pd
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, f1_score, make_scorer
from scipy.sparse import coo_matrix, hstack
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
import lightgbm as lgbm

from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping
from keras import metrics

In [219]:
scorer = make_scorer(f1_score)

In [171]:
df = pd.read_csv('texts.csv')
df.head()

,date,text
0,09.11.2017,Компания рассчитывает на решение по газовому с...
1,08.11.2017,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,01.11.2017,Новая редакция американских санкций ставит по...
3,30.10.2017,"Как стало известно “Ъ”, известный на рынке ри..."
4,23.10.2017,"НОВАТЭК, который через пять лет собирается за..."


In [172]:
pr_all = pd.read_csv('gazprom_prices.csv', sep=';')
pr_all.columns = [i.lower() for i in pr_all.columns]
pr_all.head(10)

,date,open,closingprice,dailyhigh,dailylow,volumepcs
0,08.12.2017,"133,43000","132,60000","133,90000","132,00000",16037970
1,07.12.2017,"133,70000","133,02000","133,87000","132,81000",18198430
2,06.12.2017,"133,33000","134,00000","134,29000","132,91000",14641730
3,05.12.2017,"133,48000","133,65000","133,99000","132,78000",12684800
4,04.12.2017,"133,01000","133,77000","134,00000","131,93000",17818980
5,01.12.2017,"132,49000","133,02000","133,32000","131,72000",24755830
6,30.11.2017,"133,00000","132,15000","134,31000","132,00000",40024830
7,29.11.2017,"134,85000","133,55000","134,86000","132,97000",27263040
8,28.11.2017,"133,23000","135,18000","135,18000","132,55000",26663710
9,27.11.2017,"133,69000","133,50000","135,19000","132,80000",27713150


In [173]:
pr_all.dtypes

date            object
open            object
closingprice    object
dailyhigh       object
dailylow        object
volumepcs        int64
dtype: object

Переведем все objects в float.

In [174]:
pr_all['open'] = pr_all['open'].apply(lambda a: str(a).replace(',', '.'))
pr_all['closingprice'] = pr_all['closingprice'].apply(lambda a: str(a).replace(',', '.'))
pr_all['dailyhigh'] = pr_all['dailyhigh'].apply(lambda a: str(a).replace(',', '.'))
pr_all['dailylow'] = pr_all['dailylow'].apply(lambda a: str(a).replace(',', '.'))

pr_all[['open', 'closingprice', 'dailyhigh', 'dailylow']] = pr_all[['open', 'closingprice', 'dailyhigh', 'dailylow']].astype('float')

In [175]:
pr_all.dtypes

date             object
open            float64
closingprice    float64
dailyhigh       float64
dailylow        float64
volumepcs         int64
dtype: object

## Часть 1. Вводная [3 балла]

Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. Ответьте на следующие вопросы:
* Есть ли корреляция между средней длинной текста за день и ценой закрытия?
* Есть ли корреляция между количеством упоминаний Алексея Миллера  и ценой закрытия? Учтите разные варианты написания имени.
* Упоминаний какого газопровода в статьях больше: 
    * "северный поток"
    * "турецкий поток"?
* Кого упоминают чаще:
    * Алексея Миллера
    * Владимира Путина?
* О каких санкциях пишут в статьях?

In [176]:
def beautify_text(s):
    s = re.sub("[^а-яА-Я0-9]", " ", s.lower())
    s = s.replace('\n', '')
    return s

def preprocess_text(text):

    mystem = Mystem()
    sentc = [word_tokenize((beautify_text(i))) for i in text]
    filtered_sentc = [i for i in sentc if i not in stopwords.words('russian')]
    lemmatized_sentc = [[mystem.lemmatize(j)[0] for j in i] for i in sentc]
    
    return lemmatized_sentc

In [177]:
text_pr = preprocess_text(df.text)

In [178]:
df['text_preproc'] = [' '.join(i) for i in text_pr]
df['text_len'] = df.text.apply(lambda a: len(a))
df['miller'] = [' '.join(i).count('алексей миллер') for i in text_pr]

In [179]:
pr_all = pr_all.sort_values(by=['date'])
df = df.sort_values(by=['date'])

In [180]:
gasprom_info = pd.merge(pr_all, df, on=['date'])
gasprom_info.head(10)

,date,open,closingprice,dailyhigh,dailylow,volumepcs,text,text_preproc,text_len,miller
0,01.02.2010,184.74,189.85,190.40,183.50,76298175,"""Газпром"" не исключает в 2010 г. выпуска обли...",газпром не исключать в 2010 г выпуск облигация...,256,0
1,01.02.2011,198.41,204.91,205.00,197.80,87981195,На российском ТВ — вновь дефицит рекламного в...,на российский тв вновь дефицит рекламный время...,586,0
2,01.02.2012,183.00,185.54,186.75,182.60,44145020,Федеральная антимонопольная служба (ФАС) приз...,федеральный антимонопольный служба фас признав...,857,0
3,01.02.2013,142.45,142.41,143.47,141.87,27154010,Правительство выдвинуло 14 кандидатов на 11 м...,правительство выдвигать 14 кандидат на 11 мест...,171,0
4,01.02.2016,136.01,133.90,136.34,132.82,31931470,"""Газпром"" не исключил участия в реализации эк...",газпром не исключать участие в реализация эксп...,1224,0
5,01.02.2017,150.00,149.65,150.38,148.32,20916550,Сегодня исследовательская компания Brand Fina...,сегодня исследовательский компания опубликовыв...,972,0
6,01.03.2012,192.53,194.01,194.27,191.76,31594230,"""Газпром"" скорректирует условия поставок росс...",газпром скорректировать условие поставка росси...,983,2
7,01.03.2017,133.50,133.85,134.99,133.00,38131650,Правление «Газпрома» предложило сохранить ди...,правление газпром предлагать сохранять дивиден...,1167,0
8,01.04.2014,135.90,135.89,136.73,133.84,64684830,Moody's Investors Service поставило рейтинги ...,поставлять рейтинг оао газпром и оао роснефть ...,1398,0
9,01.04.2016,147.02,147.20,147.50,143.93,36517160,"""Газпром-медиа"", управляющий телеканалами ТНТ...",газпром медиа управлять телеканал тнт нтв ради...,789,0


In [181]:
gasprom_info.corr()['text_len']['closingprice']

0.014158394621268122

In [182]:
gasprom_info.corr()['miller']['closingprice']

0.0032372019497298519

In [183]:
print('Северный поток:', sum([' '.join(i).count('северный поток') for i in text_pr]), 
'Турецкий поток:', sum([' '.join(i).count('турецкий поток') for i in text_pr]))

Северный поток: 15 Турецкий поток: 39


In [184]:
print('Алексей Миллер:', sum([' '.join(i).count('алексей миллер') for i in text_pr]), 
'Владимир Путин:', sum([' '.join(i).count('владимир путин') for i in text_pr]))

Алексей Миллер: 125 Владимир Путин: 67


In [185]:
sanc_set = set()
for i in text_pr:
    if 'санкция' in i:
        for j in i:
            if j == 'санкция':
                break
            sanc = j
        if sanc[-2:] in ['ий', 'ой', 'ый']:
            sanc_set.add(sanc)
            
print('Санкции в статьях:', sanc_set)

Санкции в статьях: {'персональный', 'очередной', 'антироссийский', 'западный', 'финансовый', 'экономический', 'штрафной', 'американский', 'международный'}


## Часть 2. Классификационная [3 балла]
Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится цена закрытия.
Для этого:
* бинаризуйте признак "цена закрытия":  новый признак ClosingPrice_bin равен 1, если по сравнению со вчера цена не упала, и 0 – в обратном случае;
* составьте обучающее и тестовое множество: данные до начала 2016 года используются для обучения, данные с 2016 года и позже – для тестирования.

Таким образом, в каждлый момент времени мы знаем: 
* ClosingPrice_bin – бинарый целевой признак
* слова из статей, опубликованных в этот день – объясняющие признаки

В этой части задания вам нужно сделать baseline алгоритм и попытаться его улучшить в следующей части. 

Используйте любой известный вам алгоритм классификации текстов для того, Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации.
Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации.

In [186]:
gasprom_info.loc[0, 'closingprice_bin'] = 0
for i in range(1, len(gasprom_info)):
    gasprom_info.loc[i, 'closingprice_bin'] = (1 + (np.sign(gasprom_info.loc[i, 'closingprice'] - gasprom_info.loc[(i-1), 'closingprice']))) / 2

In [187]:
gasprom_info.head()

,date,open,closingprice,dailyhigh,dailylow,volumepcs,text,text_preproc,text_len,miller,closingprice_bin
0,01.02.2010,184.74,189.85,190.40,183.50,76298175,"""Газпром"" не исключает в 2010 г. выпуска обли...",газпром не исключать в 2010 г выпуск облигация...,256,0,0.0
1,01.02.2011,198.41,204.91,205.00,197.80,87981195,На российском ТВ — вновь дефицит рекламного в...,на российский тв вновь дефицит рекламный время...,586,0,1.0
2,01.02.2012,183.00,185.54,186.75,182.60,44145020,Федеральная антимонопольная служба (ФАС) приз...,федеральный антимонопольный служба фас признав...,857,0,0.0
3,01.02.2013,142.45,142.41,143.47,141.87,27154010,Правительство выдвинуло 14 кандидатов на 11 м...,правительство выдвигать 14 кандидат на 11 мест...,171,0,0.0
4,01.02.2016,136.01,133.90,136.34,132.82,31931470,"""Газпром"" не исключил участия в реализации эк...",газпром не исключать участие в реализация эксп...,1224,0,0.0


In [188]:
gasprom_info['date'] = pd.to_datetime(gasprom_info['date'])  

In [189]:
X_train, y_train = gasprom_info[gasprom_info['date'] <= '2016-1-1']['text_preproc'], gasprom_info[gasprom_info['date'] <= '2016-1-1']['closingprice_bin']
X_test, y_test = gasprom_info[gasprom_info['date'] >= '2016-1-1']['text_preproc'], gasprom_info[gasprom_info['date'] >= '2016-1-1']['closingprice_bin']

In [190]:
all_text = pd.concat([X_train, X_test])

In [192]:
word_vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 3),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(X_train)
test_word_features = word_vectorizer.transform(X_test)

char_vectorizer = TfidfVectorizer(
    analyzer='char',
    stop_words='english',
    ngram_range=(1, 3),
    max_features=30000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(X_train)
test_char_features = char_vectorizer.transform(X_test)

train_features = hstack((train_char_features, train_word_features))
test_features = hstack((test_char_features, test_word_features))

## LogReg

In [196]:
shuffle = StratifiedKFold(n_splits=4)
lr = LogisticRegression()
cv_score = np.mean(cross_val_score(lr, train_features, y_train, cv=shuffle, scoring='f1', n_jobs=1))
lr.fit(train_features, y_train)
logreg_train_pred = lr.predict(train_features)
logreg_test_pred = lr.predict(test_features)
print('Logreg on 10 folds:', cv_score)
print('Logreg on train:', f1_score(logreg_train_pred, y_train))
print('Logreg on test:', f1_score(logreg_test_pred, y_test))

Logreg on 10 folds: 0.645452396359
Logreg on train: 0.962436548223
Logreg on test: 0.464566929134


## SVM + hyperopt 

In [204]:
param_space = {
                'kernel': hp.choice('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
              }

def objective_svc_f1(params):
    model = SVC(kernel=params['kernel'])
    
    shuffle = KFold()
    score = cross_val_score(model, train_features, y_train, cv=shuffle, scoring='f1', n_jobs=1)
    return 1-score.mean()

trials = Trials()

print('Fitting model...')
best_svm = fmin(objective_svc_f1,
            param_space,
            algo=tpe.suggest,
            max_evals=10)
print('best f1 param:', best_svm)


Fitting model...
best f1 param: {'kernel': 2}


In [205]:
svm = SVC(kernel='rbf')
cv_score = np.mean(cross_val_score(svm, train_features, y_train, cv=shuffle, scoring='f1'))
svm.fit(train_features, y_train)
svm_train_pred = svm.predict(train_features)
svm_test_pred = svm.predict(test_features)
print('Best svm on 10 folds:', cv_score)
print('Best svm on train:', f1_score(svm_train_pred, y_train))
print('Best svm on test:', f1_score(svm_test_pred, y_test))

Best svm on 10 folds: 0.693255313283
Best svm on train: 0.693255982596
Best svm on test: 0.521489971347


## Gradient Boosting

In [206]:
def objective_lgbm(params):
    params = {
        'num_leaves': int(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    model = lgbm.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.01,
        **params
    )
    shuffle = KFold()
    score = cross_val_score(model, train_features, y_train, cv=shuffle, scoring='f1', n_jobs=1)
    return 1-score.mean()

trials = Trials()

In [207]:
param_space = {
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
}

print('Fitting model...')
best_lgbm = fmin(objective_lgbm,
            param_space,
            algo=tpe.suggest,
            max_evals=4,
            trials=trials)
print('best:', best_lgbm)

Fitting model...
best: {'num_leaves': 30.0, 'colsample_bytree': 0.5049119421032009}


In [211]:
lgbm_c = lgbm.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.01,
        num_leaves=30 ,
        colsample_bytree=0.5049119421032009 )

cv_score = np.mean(cross_val_score(lgbm_c, train_features, y_train, cv=10, scoring='f1'))
lgbm_c.fit(train_features, y_train)
lgbm_train_pred = lgbm_c.predict(train_features)
lgbm_test_pred = lgbm_c.predict(test_features)
print('Best lgbm on 10 folds:', cv_score)
print('Best lgbm on train:', f1_score(lgbm_train_pred, y_train))
print('Best lgbm on test:', f1_score(lgbm_test_pred, y_test))

Best svm on 10 folds: 0.584005967598
Best svm on train: 1.0
Best svm on test: 0.463414634146


/home/bobrg/anaconda3/lib/python3.5/site-packages/lightgbm/basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


## Simple MLP

In [214]:
train_features.shape

(901, 18322)

In [272]:
from keras.optimizers import Adam

In [286]:
def get_simple_model():
    model = Sequential()
    model.add(Dense(1024, activation='relu', input_shape=(18322, )))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy', metrics.binary_accuracy])
    return model
model = get_simple_model()
model.fit(train_features.todense(), y_train, batch_size=4, epochs=5, verbose=1, validation_split=0.2, callbacks=[EarlyStopping()])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 1024)              18762752  
_________________________________________________________________
dropout_37 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_57 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_38 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_58 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_39 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 257       
Total para

In [287]:
f1_score(model.predict_classes(test_features.todense()).reshape(258), y_test)

0.52148997134670494

In [288]:
np.mean(model.predict_classes(test_features.todense()).reshape(258) == y_test)

0.35271317829457366

## Часть 3. Творческая [4 балла]
Придумайте и попытайтесь сделать еще что-нибудь, чтобы улучшить качество классификации. 
Направления развития:
* Морфологический признаки: 
    * использовать в качестве признаков только существительные или только именованные сущности;
* Модели скрытых тем:
    * использовать в качестве признаков скрытые темы;
    * использовать в качестве признаков динамические скрытые темы 
    пример тут: (https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb)
* Синтаксические признаки:
    * использовать SOV-тройки в качестве признаков
    * кластеризовать SOV-тройки по усредненным эмбеддингам  (обученные word2vec модели можно скачать отсюда: (http://rusvectores.org/ru/models/ или https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md) и использовать только центроиды кластеров в качестве признаков
* что-нибудь еще     

In [308]:
MAX_SEQUENCE_LENGTH = 500
MAX_FEATURES = 10000
EMBEDDING_DIM = 300

In [322]:
print('loading word embeddings...')
embeddings_index = {}
f = open('wiki.ru.vec', encoding='utf-8')
for line in (f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...
found 0 word vectors


In [316]:
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open('wiki.ru.vec'))

tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)

train_data = pad_sequences(sequences=tokenizer.texts_to_sequences(X_train), maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(sequences=tokenizer.texts_to_sequences(X_test), maxlen=MAX_SEQUENCE_LENGTH)
 
    
word_index = tokenizer.word_index

nb_words = min(MAX_FEATURES, len(word_index))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_FEATURES: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


In [310]:
model = Sequential()
model.add(Embedding(MAX_FEATURES, EMBEDDING_DIM))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 300)         9000000   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 1024)        922624    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 512)         1573376   
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 512)         0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               245200    
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 101       
Total params: 11,741,301
Trainable params: 11,741,301
Non-trainable params: 0
________________________________________________________________

In [311]:
model.fit(train_data, y_train, batch_size=4, epochs=5, verbose=1, validation_split=0.2, callbacks=[EarlyStopping()])

Train on 720 samples, validate on 181 samples
Epoch 1/5
720/720 [==============================] - 364s 505ms/step - loss: 0.6948 - acc: 0.5417 - val_loss: 0.6923 - val_acc: 0.5193
Epoch 2/5
720/720 [==============================] - 347s 481ms/step - loss: 0.6484 - acc: 0.5875 - val_loss: 0.7038 - val_acc: 0.4972


In [312]:
test_data.shape

(258, 500)

In [313]:
f1_score(model.predict_classes(test_data), y_test)

0.22972972972972971

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 22.03.2018. Каждый день просрочки дедлайна штрафуется -1 баллом.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 2: Таисия Глушкова (email: glushkovato@gmail.com, telegram: @glushkovato).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

